In [1]:
import pandas as pd
import numpy as np
import random

import quantlib
import quantlib.indicators_cal as indicators_cal
import quantlib.diagnostics_utils as diagnostics_utils

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_instruments_from_df(df):
    instruments = []
    for col in df.columns:
        inst = col.split(" ")[0]
        if "USD" in inst and inst not in instruments:
            instruments.append(inst)
    return instruments

origin_df = pd.read_excel("../crypto_historical_4h.xlsx", engine="openpyxl", index_col='open_time')

In [3]:
"""
In this notebook, we want to generalize the framework to explore the factor effect in crypto trading

Steps to do this task:
1. Read the crypto data, whatever the format, we still might change the format later
2. Filter data at the very beginning could be not appropriate, because there are always listings and unlistings
3. Take the index: datetime as a important thing to track trades
4. We should have a function to calculate factor, for example:
    def get_momentum(origin_df):
        return {token:{
            "score": score,
            "rank": rank
        }}
5. Before making every trade, calculate the sum of scores
6. Choose symbols to trade accordingly

"""
instruments = get_instruments_from_df(origin_df)
instruments

['BTCUSDT',
 'ETHUSDT',
 'BCHUSDT',
 'XRPUSDT',
 'EOSUSDT',
 'LTCUSDT',
 'TRXUSDT',
 'ETCUSDT',
 'LINKUSDT',
 'XLMUSDT',
 'ADAUSDT',
 'XMRUSDT',
 'DASHUSDT',
 'ZECUSDT',
 'XTZUSDT',
 'ATOMUSDT',
 'ONTUSDT',
 'IOTAUSDT',
 'BATUSDT',
 'VETUSDT',
 'NEOUSDT',
 'QTUMUSDT',
 'IOSTUSDT',
 'THETAUSDT',
 'ALGOUSDT',
 'ZILUSDT',
 'KNCUSDT',
 'ZRXUSDT',
 'COMPUSDT',
 'OMGUSDT',
 'DOGEUSDT',
 'SXPUSDT',
 'KAVAUSDT',
 'BANDUSDT',
 'RLCUSDT',
 'WAVESUSDT',
 'MKRUSDT',
 'SNXUSDT',
 'DOTUSDT',
 'DEFIUSDT',
 'YFIUSDT',
 'BALUSDT',
 'CRVUSDT',
 'RUNEUSDT',
 'SUSHIUSDT',
 'SRMUSDT',
 'EGLDUSDT',
 'SOLUSDT',
 'ICXUSDT',
 'STORJUSDT',
 'BLZUSDT',
 'UNIUSDT',
 'AVAXUSDT',
 'FTMUSDT',
 'HNTUSDT',
 'ENJUSDT',
 'FLMUSDT',
 'TOMOUSDT',
 'RENUSDT',
 'KSMUSDT',
 'NEARUSDT',
 'AAVEUSDT',
 'FILUSDT',
 'RSRUSDT',
 'LRCUSDT',
 'MATICUSDT',
 'OCEANUSDT',
 'CVCUSDT',
 'BELUSDT',
 'CTKUSDT',
 'AXSUSDT',
 'ALPHAUSDT',
 'ZENUSDT',
 'SKLUSDT',
 'GRTUSDT',
 '1INCHUSDT',
 'CHZUSDT',
 'SANDUSDT',
 'ANKRUSDT',
 'BTSUSDT',
 'LI

In [4]:
look_back = 50
ignore = 5
symbol = "BTCUSDT"

class MomentumFactor:
    """
    @Params:
        df: the original df contains all symbol information
        index: the index that a trading event should happend
        look_back: look back period to calculate the momentum
        ingore: int, to prevent short term reversing, ignore the most recent klines
    """
    def __init__(self, look_back, ignore):
        self.look_back = look_back
        self.ignore = ignore
        
    def calculate(self, df, symbol):
        close_prices = df["{} close".format(symbol)]
        return close_prices.shift(self.ignore) / close_prices.shift(self.look_back+self.ignore) - 1


In [5]:
"""
Every time we trigger trade function, we calculate all factors for all symbols
"""

def calculate_factor_scores(df, target_symbols, factors, trade_index):
    score_dict = {}
    for symbol in target_symbols:
        factor_scores = sum([factor.calculate(df, symbol)[trade_index] for factor in factors])
        score_dict[symbol] = factor_scores
    return score_dict

def trade(df, target_symbols, factors, trade_index):
    """
    @Params:
        factors: a list of factors that we want to use
        trade_index: the index that the trade is happening
    """
    score_dict = calculate_factor_scores(df, target_symbols, factors, trade_index)
    print (score_dict)

momentum = MomentumFactor(look_back, ignore)
factors = [momentum]
trade(origin_df, instruments, factors, -1)

{'BTCUSDT': 0.08017795952097684, 'ETHUSDT': 0.11885510909703023, 'BCHUSDT': 0.08367428800641807, 'XRPUSDT': 0.01900842641583389, 'EOSUSDT': 0.006504065040650486, 'LTCUSDT': 0.11423859263331493, 'TRXUSDT': 0.002114484216885648, 'ETCUSDT': 0.08070568741166739, 'LINKUSDT': 0.0983991167540712, 'XLMUSDT': 0.008018867924528328, 'ADAUSDT': 0.15242018537590107, 'XMRUSDT': 0.036537487335359664, 'DASHUSDT': 0.09225026661926772, 'ZECUSDT': 0.12783342119135477, 'XTZUSDT': 0.04900181488203259, 'ATOMUSDT': 0.14606437652150395, 'ONTUSDT': 0.12787356321839094, 'IOTAUSDT': 0.004428697962798989, 'BATUSDT': -0.0027662517289074318, 'VETUSDT': 0.08302507192766129, 'NEOUSDT': 0.10590402948009281, 'QTUMUSDT': 0.06850172036284019, 'IOSTUSDT': 0.07112146184163382, 'THETAUSDT': 0.059117010260755, 'ALGOUSDT': 0.05790441176470584, 'ZILUSDT': 0.05521472392638027, 'KNCUSDT': -0.00758054327226787, 'ZRXUSDT': 0.09246088193456603, 'COMPUSDT': 0.07594639078297671, 'OMGUSDT': -0.02022926500337152, 'DOGEUSDT': 0.07712701

In [11]:
from collections import defaultdict

def get_rankings(factor_dict, n_ranks=5):
    """
    According to factor scores in factor_dict, generate rankings for each factor
    Returns:
        dict(rank: [symbol1, symbol2])
    """
    sorted_symbols = sorted(factor_dict, key=factor_dict.get, reverse=True)

    # Calculate the rank for each symbol
    num_symbols = len(sorted_symbols)
    rank_size = num_symbols // n_ranks + (num_symbols % n_ranks > 0)
    ranks_dict = defaultdict(list)
    for i, symbol in enumerate(sorted_symbols):
        rank = i // rank_size
        ranks_dict[rank].append(symbol)

    return ranks_dict

def is_rebalancing(i, rebalance_period):
    return i!=1 and i%rebalance_period==1

def get_rank_porfolio(df, target_symbols, factor_specs, trade_index):

    rank_pofolio = []
    for i, (factor, n_ranks) in enumerate(factor_specs):
        # Calculate scores for every symbol
        factor_dict = {}
        for symbol in target_symbols:
            factor_dict[symbol] = factor.calculate(df, symbol)[trade_index]
        rank_dict = get_rankings(factor_dict, n_ranks)
        rank_pofolio.append(rank_dict)

    return rank_pofolio
    

def insert_record(dt, rank_df, rank_portfolio):
    """
    rank_df: DataFrame with columns for each rank and each instrument, as well as columns for the forward returns of each instrument
    rank_portfolio: list of rank dictionaries, where each rank dictionary maps ranks to a list of symbols

    2 factors maximum
    """
    # Create a new row for the rank details
    new_row = pd.DataFrame(index=[dt])
    
    # Iterate over each factor and its corresponding rank portfolio
    for i, rank_dict in enumerate(rank_portfolio):
        # Iterate over each rank and its corresponding symbols
        for rank, symbols in rank_dict.items():
            # Set the symbols for the corresponding rank and factor columns
            col_name = f'rank_{i+1}_{rank}'
            new_row[col_name] = ','.join(symbols)

    # Concatenate the new row to the rank_df DataFrame
    rank_df = pd.concat([rank_df, new_row])
    return rank_df
    

def analyse_factor_rank():
    pass

def trade_with_ranks():
    pass

def get_target_symbols(row):
    target_symbols = []
    for inst in instruments:
        if row[f"{inst} active"] == True:
            target_symbols.append(inst)

    return target_symbols

def _add_fwd_ret(df, period):
    all_symbols = get_instruments_from_df(df)
    for symbol in all_symbols:
        df["{} {} fwd ret".format(symbol, period)] = df["{} close".format(symbol)].shift(-period) / df["{} close".format(symbol)] - 1

    return df

def backtest(df, rebalance_period, factor_specs):
    # rank_df, every row should contains the rank details for that time
    
    df = _add_fwd_ret(df, rebalance_period)
    rank_df = pd.DataFrame()

    for i, dt in enumerate(df.index):
        if is_rebalancing(i, rebalance_period):
            target_symbols = get_target_symbols(df.iloc[i])
            rank_porfolio = get_rank_porfolio(df, target_symbols, factor_specs, trade_index=i)
            rank_df = insert_record(dt, rank_df, rank_porfolio)

    return rank_df
    #trade_with_ranks(rank_df)

momentum1 = MomentumFactor(look_back, ignore)
momentum2 = MomentumFactor(look_back=20, ignore=2)
factor_specs = [(momentum1, 5), (momentum2, 2)]
backtest(origin_df, rebalance_period=20, factor_specs=factor_specs)

,rank_1_0,rank_1_1,rank_1_2,rank_1_3,rank_1_4,rank_2_0,rank_2_1
2022-07-05 00:00:00,"BTCUSDT,ETHUSDT,BCHUSDT,XRPUSDT,EOSUSDT,LTCUSD...","ZRXUSDT,COMPUSDT,OMGUSDT,DOGEUSDT,SXPUSDT,KAVA...","HNTUSDT,ENJUSDT,FLMUSDT,TOMOUSDT,RENUSDT,KSMUS...","REEFUSDT,RVNUSDT,SFPUSDT,XEMUSDT,COTIUSDT,MANA...","1000XECUSDT,GALAUSDT,CELOUSDT,ARUSDT,KLAYUSDT,...","BTCUSDT,ETHUSDT,BCHUSDT,XRPUSDT,EOSUSDT,LTCUSD...","BELUSDT,CTKUSDT,AXSUSDT,ZENUSDT,SKLUSDT,GRTUSD..."
2022-07-08 08:00:00,"BTCUSDT,ETHUSDT,BCHUSDT,XRPUSDT,EOSUSDT,LTCUSD...","COMPUSDT,OMGUSDT,DOGEUSDT,SXPUSDT,KAVAUSDT,BAN...","FLMUSDT,TOMOUSDT,RENUSDT,KSMUSDT,NEARUSDT,AAVE...","SFPUSDT,XEMUSDT,COTIUSDT,CHRUSDT,MANAUSDT,ALIC...","CELOUSDT,ARUSDT,KLAYUSDT,ARPAUSDT,CTSIUSDT,LPT...","MATICUSDT,IOTXUSDT,GMTUSDT,CRVUSDT,AAVEUSDT,ST...","NKNUSDT,FLOWUSDT,XLMUSDT,KAVAUSDT,RSRUSDT,YFIU..."
2022-07-11 16:00:00,"DYDXUSDT,AAVEUSDT,SUSHIUSDT,ONEUSDT,SRMUSDT,UN...","JASMYUSDT,LINAUSDT,XMRUSDT,IOTAUSDT,SANDUSDT,D...","API3USDT,KNCUSDT,NEARUSDT,APEUSDT,C98USDT,DASH...","LTCUSDT,XRPUSDT,LRCUSDT,BCHUSDT,ATAUSDT,MTLUSD...","GALUSDT,DOTUSDT,LINKUSDT,CTKUSDT,AUDIOUSDT,GAL...","SRMUSDT,UNFIUSDT,COMPUSDT,ONEUSDT,XTZUSDT,AAVE...","AUDIOUSDT,CTSIUSDT,OMGUSDT,ALPHAUSDT,ATAUSDT,B..."
2022-07-15 00:00:00,"FLMUSDT,SRMUSDT,AAVEUSDT,UNIUSDT,RAYUSDT,ONEUS...","FTTUSDT,EGLDUSDT,CTKUSDT,ALGOUSDT,SNXUSDT,REEF...","TRXUSDT,DASHUSDT,XLMUSDT,AUDIOUSDT,ANKRUSDT,BT...","PEOPLEUSDT,ANTUSDT,JASMYUSDT,ALICEUSDT,ZILUSDT...","APEUSDT,1000XECUSDT,KNCUSDT,ATAUSDT,SXPUSDT,GA...","FLMUSDT,CRVUSDT,ARUSDT,AAVEUSDT,SNXUSDT,RAYUSD...","MTLUSDT,ICXUSDT,XLMUSDT,HOTUSDT,BATUSDT,ETCUSD..."
2022-07-18 08:00:00,"ARUSDT,FLMUSDT,MATICUSDT,ETCUSDT,AAVEUSDT,RAYU...","AUDIOUSDT,RENUSDT,DEFIUSDT,ONEUSDT,TOMOUSDT,SO...","BLZUSDT,HBARUSDT,XTZUSDT,DARUSDT,ATOMUSDT,SAND...","XLMUSDT,IOSTUSDT,KAVAUSDT,DUSKUSDT,LPTUSDT,LIN...","MANAUSDT,FTMUSDT,BTCDOMUSDT,ALICEUSDT,THETAUSD...","ETCUSDT,OPUSDT,RVNUSDT,NKNUSDT,GTCUSDT,BTSUSDT...","XMRUSDT,REEFUSDT,THETAUSDT,NEARUSDT,EOSUSDT,FL..."
...,...,...,...,...,...,...,...
2023-04-01 00:00:00,"SXPUSDT,XRPUSDT,MASKUSDT,C98USDT,XLMUSDT,TOMOU...","NEOUSDT,BAKEUSDT,ZENUSDT,BALUSDT,WOOUSDT,LITUS...","APEUSDT,SUSHIUSDT,ALICEUSDT,BLUEBIRDUSDT,LTCUS...","BTCDOMUSDT,RVNUSDT,GALAUSDT,UNFIUSDT,DYDXUSDT,...","CTSIUSDT,DARUSDT,OCEANUSDT,AAVEUSDT,FTMUSDT,FE...","SXPUSDT,C98USDT,HOTUSDT,CFXUSDT,LINAUSDT,HBARU...","IOSTUSDT,CHRUSDT,DOTUSDT,CTSIUSDT,ARPAUSDT,AVA..."
2023-04-04 08:00:00,"SXPUSDT,INJUSDT,ICXUSDT,C98USDT,ZRXUSDT,LINAUS...","EOSUSDT,STORJUSDT,JASMYUSDT,BANDUSDT,DOTUSDT,B...","YFIUSDT,ETHUSDT,IDUSDT,DENTUSDT,AAVEUSDT,TRXUS...","DODOBUSD,ETCUSDT,BLZUSDT,MASKUSDT,STMXUSDT,NEO...","LRCUSDT,OPUSDT,SANDUSDT,APTUSDT,1000LUNCUSDT,X...","ICXUSDT,DOGEUSDT,SXPUSDT,INJUSDT,IOTXUSDT,LQTY...","AXSUSDT,EOSUSDT,FILUSDT,STORJUSDT,MKRUSDT,ONEU..."
2023-04-07 16:00:00,"SXPUSDT,ICXUSDT,LINAUSDT,INJUSDT,ONTUSDT,ZRXUS...","IOTAUSDT,ENJUSDT,WOOUSDT,CRVUSDT,BAKEUSDT,ATAU...","RVNUSDT,SFPUSDT,DARUSDT,BLZUSDT,JASMYUSDT,GMXU...","XTZUSDT,CHRUSDT,SANDUSDT,DENTUSDT,ETCUSDT,API3...","GMTUSDT,IDUSDT,RLCUSDT,BTCDOMUSDT,SOLUSDT,CFXU...","LINAUSDT,ONTUSDT,ACHUSDT,ENJUSDT,STGUSDT,ANTUS...","ONEUSDT,NEOUSDT,EGLDUSDT,RNDRUSDT,QTUMUSDT,ZEC..."
2023-04-11 00:00:00,"NKNUSDT,ANTUSDT,LINAUSDT,BELUSDT,ACHUSDT,ICXUS...","KNCUSDT,BNXUSDT,GMXUSDT,MKRUSDT,CVXUSDT,TLMUSD...","BCHUSDT,BTCUSDT,SANDUSDT,SKLUSDT,XRPUSDT,DARUS...","IOSTUSDT,1000LUNCUSDT,DOTUSDT,APTUSDT,ASTRUSDT...","CHRUSDT,WOOUSDT,MTLUSDT,EGLDUSDT,DASHUSDT,COTI...","LEVERUSDT,NKNUSDT,CFXUSDT,STXUSDT,JASMYUSDT,BE...","SUSHIUSDT,OGNUSDT,EOSUSDT,FTMUSDT,SFPUSDT,OPUS..."
